In [1]:
from loader import MoleculeDataset, SeqDataset,SeqMolDataset#########################
import torch
import torch
from torchvision.models import resnet18

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import argparse

from loader import MoleculeDataset#################
#from torch_geometric.data import DataLoader
from torch_geometric.loader import DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm import tqdm
import numpy as np

from model import GNN, GNN_graphpred,GNN_graphpred_1
from sklearn.metrics import roc_auc_score

from splitters import scaffold_split,scaffold_split_1
import pandas as pd
import os
import shutil
from tensorboardX import SummaryWriter
import esm

from SeqMolModel import InteractionModel,InteractionModel_1

In [2]:
# Training settings
parser = argparse.ArgumentParser(description='PyTorch implementation of pre-training of graph neural networks')
parser.add_argument('--device', type=int, default=0,
                        help='which gpu to use if any (default: 0)')#0000
parser.add_argument('--batch_size', type=int, default=16,
                        help='input batch size for training (default: 32)')
parser.add_argument('--epochs', type=int, default=100,
                        help='number of epochs to train (default: 100)')
parser.add_argument('--lr', type=float, default=0.001,
                        help='learning rate (default: 0.001)')
parser.add_argument('--lr_scale', type=float, default=1,
                        help='relative learning rate for the feature extraction layer (default: 1)')
parser.add_argument('--decay', type=float, default=0,
                        help='weight decay (default: 0)')
parser.add_argument('--num_layer', type=int, default=5,
                        help='number of GNN message passing layers (default: 5).')
parser.add_argument('--emb_dim', type=int, default=300,
                        help='embedding dimensions (default: 300)')
parser.add_argument('--dropout_ratio', type=float, default=0.5,
                        help='dropout ratio (default: 0.5)')
parser.add_argument('--graph_pooling', type=str, default="mean",
                        help='graph level pooling (sum, mean, max, set2set, attention)')
parser.add_argument('--JK', type=str, default="last",
                        help='how the node features across layers are combined. last, sum, max or concat')
parser.add_argument('--gnn_type', type=str, default="gin")
parser.add_argument('--dataset', type=str, default = 'sider', help='root directory of dataset. For now, only classification.')
#parser.add_argument('--input_model_file', type=str, default = 'None', help='filename to read the model (if there is any)')
parser.add_argument('--input_model_file', type=str, default = 'Mole-BERT', help='filename to read the model (if there is any)')
parser.add_argument('--filename', type=str, default = '', help='output filename')
parser.add_argument('--seed', type=int, default=42, help = "Seed for splitting the dataset.")
parser.add_argument('--runseed', type=int, default=0, help = "Seed for minibatch selection, random initialization.")
parser.add_argument('--split', type = str, default="scaffold", help = "random or scaffold or random_scaffold")
parser.add_argument('--eval_train', type=int, default = 1, help='evaluating training or not')
parser.add_argument('--num_workers', type=int, default = 4, help='number of workers for dataset loading')
args = parser.parse_args(args=[])###############33


In [3]:
device = torch.device("cuda:" + str(args.device)) if torch.cuda.is_available() else torch.device("cpu")
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.runseed)

num_tasks=1
# Load ESM-2 model
protein_model, protein_alphabet = esm.pretrained.esm2_t33_650M_UR50D()

print(protein_alphabet)
#alphabet = esm.Alphabet.from_architecture(model_data["args"].arch)
#batch_converter = alphabet.get_batch_converter()
protein_model.eval()  # disables dropout for deterministic results

#self.molecular.model,self.molecular.node_representation,self.molecular.features = GNN_graphpred_1(args.num_layer, args.emb_dim, num_tasks, JK = args.JK, drop_ratio = args.dropout_ratio, graph_pooling = args.graph_pooling, gnn_type = args.gnn_type)
molecular_model = GNN_graphpred(args.num_layer, args.emb_dim, num_tasks, JK = args.JK, drop_ratio = args.dropout_ratio, graph_pooling = args.graph_pooling, gnn_type = args.gnn_type)
###################################
if not args.input_model_file == "None":###############
    print('Not from scratch')
    molecular_model.from_pretrained('model_gin/{}.pth'.format(args.input_model_file))
    print('rese:model_gin')
molecular_model.to(device)



model_param_group = []
model_param_group.append({"params": molecular_model.gnn.parameters()})
if args.graph_pooling == "attention":
    model_param_group.append({"params": molecular_model.pool.parameters(), "lr":args.lr*args.lr_scale})
'''
————————————————
版权声明：本文为CSDN博主「Dreamcatcher风」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/Wind_2028/article/details/120541017   
'''



model_param_group.append({"params": molecular_model.graph_pred_linear.parameters(), "lr":args.lr*args.lr_scale})
optimizer = optim.Adam(model_param_group, lr=0.01, weight_decay=args.decay)#############

Not from scratch
rese:model_gin


In [4]:
#num_tasks=1
model=InteractionModel_1(protein_model=protein_model,molecular_model=molecular_model,mol_emb_dim=128,pro_emb_dim=1028,num_tasks=1) 
model.to(device)
print(model)#nice

InteractionModel_1(
  (mole_model): GNN(
    (x_embedding1): Embedding(120, 300)
    (x_embedding2): Embedding(3, 300)
    (gnns): ModuleList(
      (0): GINConv()
      (1): GINConv()
      (2): GINConv()
      (3): GINConv()
      (4): GINConv()
    )
    (batch_norms): ModuleList(
      (0): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (4): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (pred_linear): Linear(in_features=1156, out_features=1, bias=True)
)


In [5]:
gnn_dataset = MoleculeDataset("./dataset/" + args.dataset, dataset=args.dataset)###########################转换成了分子图的格式

In [6]:
seq_dataset=SeqDataset('dataset/affinity/processed/sequence.csv')
#seq_dataset[2]

In [7]:
seq_gnn_dataset=SeqMolDataset(seq_dataset,gnn_dataset)

In [ ]:
#template from chartGPT


import torch
from torch.utils.data import Dataset, DataLoader

class MyDataLoader:
    def __init__(self, dataset1, dataset2, batch_size=1, shuffle=False, num_workers=0):
        self.dataset1 = dataset1
        self.dataset2 = dataset2
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.num_workers = num_workers

    def __iter__(self):
        # Create DataLoader for each dataset
        dataloader1 = DataLoader(self.dataset1, batch_size=self.batch_size, shuffle=self.shuffle, num_workers=self.num_workers)
        dataloader2 = DataLoader(self.dataset2, batch_size=self.batch_size, shuffle=self.shuffle, num_workers=self.num_workers)

        # Iterate over batches from both datasets simultaneously
        for data1, data2 in zip(dataloader1, dataloader2):
            yield data1, data2

    def __len__(self):
        return min(len(self.dataset1), len(self.dataset2))

# Define your datasets here (subclass of torch.utils.data.Dataset)
class MyDataset1(Dataset):
    # ... (implement necessary methods)

class MyDataset2(Dataset):
    # ... (implement necessary methods)

# Example usage
if __name__ == "__main__":
    dataset1 = MyDataset1()
    dataset2 = MyDataset2()

    batch_size = 32
    shuffle = True
    num_workers = 4

    dataloader = MyDataLoader(dataset1, dataset2, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

    for batch_data1, batch_data2 in dataloader:
        # Process your batch data here
        print("Data from Dataset 1:", batch_data1)
        print("Data from Dataset 2:", batch_data2)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class MyDataLoader:
    def __init__(self, dataset1, dataset2, batch_size=1, shuffle=False, num_workers=0):
        self.dataset1 = dataset1
        self.dataset2 = dataset2
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.num_workers = num_workers

    def __iter__(self):
        # Create DataLoader for each dataset
        dataloader1 = DataLoader(self.dataset1, batch_size=self.batch_size, shuffle=self.shuffle, num_workers=self.num_workers)
        dataloader2 = DataLoader(self.dataset2, batch_size=self.batch_size, shuffle=self.shuffle, num_workers=self.num_workers)

        # Iterate over batches from both datasets simultaneously
        for data1, data2 in zip(dataloader1, dataloader2):
            yield data1, data2

    def __len__(self):
        return min(len(self.dataset1), len(self.dataset2))
'''
# Define your datasets here (subclass of torch.utils.data.Dataset)
class MyDataset1(Dataset):
    # ... (implement necessary methods)

class MyDataset2(Dataset):
    # ... (implement necessary methods)
'''
# Example usage
if __name__ == "__main__":
    dataset1 =seq_dataset
    dataset2 = gnn_dataset

    batch_size = 32
    shuffle = True
    num_workers = 4

    dataloader = MyDataLoader(dataset1, dataset2, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

    for batch_data1, batch_data2 in dataloader:
        # Process your batch data here
        print("Data from Dataset 1:", batch_data1)
        #print("Data from Dataset 2:", batch_data2)

In [7]:
'''
seq_gnn_dataset=SeqMolDataset(seq_dataset,gnn_dataset)##########required
seq_dataset=seq_gnn_dataset.datasetA
mol_dataset=seq_gnn_dataset.datasetB

seq_dataloader=DataLoader(seq_dataset,batch_size=args.batch_size,num_workers=args.num_workers,shuffle=False)
mol_dataloader=DataLoader(mol_dataset,batch_size=args.batch_size,num_workers=args.num_workers,shuffle=False)

for i,(seq,mol) in enumerate(zip(seq_dataloader,mol_dataloader)):
    print(i,seq,mol)
    
'''

key: ykey:key: key:
 yy item,slices:y
 

item,slices:item,slices: tensor([ 1,  1, -1,  ...,  1,  1,  1])  item,slices:tensor([    0,    27,    54,  ..., 38475, 38502, 38529])tensor([ 1,  1, -1,  ...,  1,  1,  1])tensor([ 1,  1, -1,  ...,  1,  1,  1])  
 tensor([    0,    27,    54,  ..., 38475, 38502, 38529])tensor([    0,    27,    54,  ..., 38475, 38502, 38529])key:
 key:xtensor([ 1,  1, -1,  ...,  1,  1,  1])

 key: item,slices:x  tensor([    0,    27,    54,  ..., 38475, 38502, 38529])

tensor([[5, 0],
        [5, 0],
        [6, 0],
        ...,
        [5, 0],
        [5, 0],
        [5, 0]])xitem,slices: key:
tensor([    0,    13,    42,  ..., 47949, 47983, 48006]) item,slices: 
 xtensor([[5, 0],
        [5, 0],
        [6, 0],
        ...,
        [5, 0],
        [5, 0],
        [5, 0]])key:tensor([[5, 0],
        [5, 0],
        [6, 0],
        ...,
        [5, 0],
        [5, 0],
        [5, 0]]) 
  item,slices:edge_indextensor([    0,    13,    42,  ..., 47949, 47983, 48006]

In [60]:
class SeqMolDataLoader_2(DataLoader):
    def __init__(self, dataset,batch_size,shuffle,num_workers):
        #super(SeqMolDataLoader, self).__init__()
        self.dataloaderA=DataLoader(seq_gnn_dataset.datasetA,batch_size,shuffle,num_workers)
        self.dataloaderB=DataLoader(seq_gnn_dataset.datasetB,batch_size,shuffle,num_workers)
        #self.data,self.slices=datasetB
    def __getitem__(self,idx):
        seqloader=self.dataloaderA[idx]
        '''
        data = Data()
        # data_masked = self.transform(data)
        # print('data', self.data.edge_index)
        # print('slices', self.slices)
        for key in self.data.keys:
            print('key:',key)
            item, slices = self.data[key], self.slices[key]
            print('item,slices:',item,slices)
            s = list(repeat(slice(None), item.dim()))
            s[data.__cat_dim__(key, item)] = slice(slices[idx],
                                                    slices[idx + 1])
            data[key] = item[s]
       '''
        molloader=self.dataloaderB[idx]
        return seqloader,molloader
    def __len__(self):
        return len(self.dataloaderA)

seq_gnn_dataset=SeqMolDataset(seq_dataset,gnn_dataset)
seq_dataset=seq_gnn_dataset.datasetA
mol_dataset=seq_gnn_dataset.datasetB
#seq_dataloader=DataLoader(seq_dataset)
#mol_dataloader=DataLoader(mol_dataset)
seq=seq_gnn_dataset.datasetA[3]
#print(seq)
data=seq_gnn_dataset.datasetB[3]
#print(data)
seq_mol_dataloader=SeqMolDataLoader_2(seq_gnn_dataset,batch_size=args.batch_size,shuffle=True,num_workers=args.num_workers)
seq_dataloader=seq_mol_dataloader.dataloaderA
seq_dataloader=seq_mol_dataloader.dataloaderB
#seq_dataloader=DataLoader(seq_dataset)
#mol_dataloader=DataLoader(mol_dataset)



    
    
class SeqMolDataLoader(DataLoader):
    def __init__(self, dataloaderA,dataloaderB):
        #super(SeqMolDataLoader, self).__init__()
        self.dataloaderA=dataloaderA
        self.dataloaderB=dataloaderB
        #self.data,self.slices=datasetB
    def __getitem__(self,idx):
        seqloader=dataloaderA[idx]
        '''
        data = Data()
        # data_masked = self.transform(data)
        # print('data', self.data.edge_index)
        # print('slices', self.slices)
        for key in self.data.keys:
            print('key:',key)
            item, slices = self.data[key], self.slices[key]
            print('item,slices:',item,slices)
            s = list(repeat(slice(None), item.dim()))
            s[data.__cat_dim__(key, item)] = slice(slices[idx],
                                                    slices[idx + 1])
            data[key] = item[s]
       '''
        molloader=dataloaderB[idx]
        return seqloader,molloader
    def __len__(self):
        return len(dataloaderA)
#seq_mol_dataloader=SeqMolDataLoader_2(seq_dataloader,mol_dataloader)        

for i, (seq, mol) in enumerate(zip(seq_dataloader, mol_dataloader)):
    print(i, seq, mol)



key: y
item,slices: tensor([ 1,  1, -1,  ...,  1,  1,  1]) tensor([    0,    27,    54,  ..., 38475, 38502, 38529])
key: id
item,slices: tensor([   0,    1,    2,  ..., 1424, 1425, 1426]) tensor([   0,    1,    2,  ..., 1425, 1426, 1427])
key: x
item,slices: tensor([[5, 0],
        [5, 0],
        [6, 0],
        ...,
        [5, 0],
        [5, 0],
        [5, 0]]) tensor([    0,    13,    42,  ..., 47949, 47983, 48006])
key: edge_index
item,slices: tensor([[ 0,  1,  1,  ..., 11, 22, 17],
        [ 1,  0,  2,  ..., 16, 17, 22]]) tensor([     0,     24,     86,  ..., 100788, 100864, 100912])
key: edge_attr
item,slices: tensor([[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [3, 0],
        [3, 0],
        [3, 0]]) tensor([     0,     24,     86,  ..., 100788, 100864, 100912])
key: y
item,slices: tensor([ 1,  1, -1,  ...,  1,  1,  1]) tensor([    0,    27,    54,  ..., 38475, 38502, 38529])
key: id
item,slices: tensor([   0,    1,    2,  ..., 1424, 1425, 1426]) tensor([   0

TypeError: 'int' object is not iterable

In [40]:
seq_gnn_dataset=SeqMolDataset(seq_dataset,gnn_dataset)
seq_dataset=seq_gnn_dataset.datasetA
mol_dataset=seq_gnn_dataset.datasetB
seq=seq_gnn_dataset.datasetA[3]
#print(seq)
data=seq_gnn_dataset.datasetB[3]
#print(data)
seq_mol_dataloader=DataLoader(seq_gnn_dataset)
seq_dataloader=DataLoader(seq_dataset)
mol_dataloader=DataLoader(mol_dataset)
class SeqMolDataLoader(DataLoader):
    def __init__(self, datasetA,datasetB,batch_size,shuffle,num_workers):
        #super(SeqMolDataLoader, self).__init__()
        #super(SeqMolDataLoader,self).__init__(datasetB)
        
        
        self.dataloaderA=DataLoader(datasetA,batch_size,shuffle,num_workers)
        self.dataloaderB=DataLoader(datasetB,batch_size,shuffle,num_workers)
        self.dataA_list=[]
        self.dataB_list=[]
        for i,(dataA,dataB) in enumerate(zip(self.dataloaderA,self.dataloaderB)):
            self.dataA_list.append(dataA)
            self.dataB_list.append(dataB)
        
        #self.data,self.slices=datasetB
    def __getitem__(self,idx):
        seqloader=self.dataA_list[idx]
        '''
        data = Data()
        # data_masked = self.transform(data)
        # print('data', self.data.edge_index)
        # print('slices', self.slices)
        for key in self.data.keys:
            print('key:',key)
            item, slices = self.data[key], self.slices[key]
            print('item,slices:',item,slices)
            s = list(repeat(slice(None), item.dim()))
            s[data.__cat_dim__(key, item)] = slice(slices[idx],
                                                    slices[idx + 1])
            data[key] = item[s]
       '''
        molloader=self.dataB_list[idx]
        return seqloader,molloader
    def __len__(self):
        return len(self.dataloaderA)
    
    
    
    
class SeqMolDataLoader_1(DataLoader):
    def __init__(self, dataset,batch_size,shuffle,num_workers):
        #super(SeqMolDataLoader, self).__init__()
        #super(SeqMolDataLoader,self).__init__(datasetB)
        self.dataloaderA=DataLoader(dataset.datasetA,batch_size,shuffle,num_workers)
        self.dataloaderB=DataLoader(dataset.datasetB,batch_size,shuffle,num_workers)
        #self.data,self.slices=datasetB
    def __getitem__(self,idx):
        seqloader=self.dataloaderA[idx]
        '''
        data = Data()
        # data_masked = self.transform(data)
        # print('data', self.data.edge_index)
        # print('slices', self.slices)
        for key in self.data.keys:
            print('key:',key)
            item, slices = self.data[key], self.slices[key]
            print('item,slices:',item,slices)
            s = list(repeat(slice(None), item.dim()))
            s[data.__cat_dim__(key, item)] = slice(slices[idx],
                                                    slices[idx + 1])
            data[key] = item[s]
       '''
        molloader=self.dataloaderB[idx]
        return seqloader,molloader
    def __len__(self):
        return len(self.dataloaderA)
    
seq_mol_dataloader=SeqMolDataLoader(seq_dataset,mol_dataset,batch_size=args.batch_size,shuffle=False,num_workers=args.num_workers)        
  
for i, (seq, mol) in enumerate(zip(seq_mol_dataloader.dataloaderA, seq_mol_dataloader.dataloaderB)):
    print(i, seq, mol)


key: y
item,slices: tensor([ 1,  1, -1,  ...,  1,  1,  1]) tensor([    0,    27,    54,  ..., 38475, 38502, 38529])
key: id
item,slices: tensor([   0,    1,    2,  ..., 1424, 1425, 1426]) tensor([   0,    1,    2,  ..., 1425, 1426, 1427])
key: x
item,slices: tensor([[5, 0],
        [5, 0],
        [6, 0],
        ...,
        [5, 0],
        [5, 0],
        [5, 0]]) tensor([    0,    13,    42,  ..., 47949, 47983, 48006])
key: edge_index
item,slices: tensor([[ 0,  1,  1,  ..., 11, 22, 17],
        [ 1,  0,  2,  ..., 16, 17, 22]]) tensor([     0,     24,     86,  ..., 100788, 100864, 100912])
key: edge_attr
item,slices: tensor([[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [3, 0],
        [3, 0],
        [3, 0]]) tensor([     0,     24,     86,  ..., 100788, 100864, 100912])
key: y
item,slices: tensor([ 1,  1, -1,  ...,  1,  1,  1]) tensor([    0,    27,    54,  ..., 38475, 38502, 38529])
key: id
item,slices: tensor([   0,    1,    2,  ..., 1424, 1425, 1426]) tensor([   0

TypeError: 'int' object is not iterable

In [13]:
if args.split == "scaffold":
        smiles_list = pd.read_csv('./dataset/' + args.dataset + '/processed/smiles.csv', header=None)[0].tolist()
        
        #print('smiles_list:',smiles_list)
        mol_train_dataset, mol_valid_dataset, mol_test_dataset ,seq_train_dataset,seq_valid_dataset,seq_test_dataset= scaffold_split_1(seq_gnn_dataset, smiles_list, null_value=0, frac_train=0.8,frac_valid=0.1, frac_test=0.1)##########dataset
        print("scaffold")
elif args.split == "random":
        train_dataset, valid_dataset, test_dataset = random_split(dataset, null_value=0, frac_train=0.8,frac_valid=0.1, frac_test=0.1, seed = args.seed)
        print("random")
elif args.split == "random_scaffold":
        smiles_list = pd.read_csv('./dataset/' + args.dataset + '/processed/smiles.csv', header=None)[0].tolist()
        train_dataset, valid_dataset, test_dataset = random_scaffold_split(dataset, smiles_list, null_value=0, frac_train=0.8,frac_valid=0.1, frac_test=0.1, seed = args.seed)
        print("random scaffold")
else:
        raise ValueError("Invalid split option.")

print('++++++++++', mol_train_dataset[0])


seq_mol_train_dataset=SeqMolDataset(seq_train_dataset,mol_train_dataset)
seq_mol_valid_dataset=SeqMolDataset(seq_valid_dataset,mol_valid_dataset)
seq_mol_test_dataset=SeqMolDataset(seq_train_dataset,mol_test_dataset)


seq_mol_train_loader = SeqMolDataLoader(seq_train_dataset,mol_train_dataset,batch_size=args.batch_size, shuffle=False, num_workers = args.num_workers)
seq_mol_val_loader = SeqMolDataLoader(seq_valid_dataset,mol_valid_dataset, batch_size=args.batch_size, shuffle=False, num_workers = args.num_workers)
seq_mol_test_loader = SeqMolDataLoader(seq_test_dataset,mol_test_dataset,batch_size=args.batch_size, shuffle=False, num_workers = args.num_workers)

for i ,(A,B) in enumerate(zip(seq_mol_train_loader.dataloaderA,seq_mol_train_loader.dataloaderB)):
    print(A)
'''
seq_mol_train_loader = SeqMolDataLoader_1(seq_mol_train_dataset,batch_size=args.batch_size, shuffle=False, num_workers = args.num_workers)
seq_mol_val_loader = SeqMolDataLoader_1(seq_mol_valid_dataset, batch_size=args.batch_size, shuffle=False, num_workers = args.num_workers)
seq_mol_test_loader = SeqMolDataLoader_1(seq_mol_test_dataset,batch_size=args.batch_size, shuffle=False, num_workers = args.num_workers)

'''

scaffold
key: y
item,slices: tensor([ 1,  1, -1,  ...,  1,  1,  1]) tensor([    0,    27,    54,  ..., 38475, 38502, 38529])
key: x
item,slices: tensor([[5, 0],
        [5, 0],
        [6, 0],
        ...,
        [5, 0],
        [5, 0],
        [5, 0]]) tensor([    0,    13,    42,  ..., 47949, 47983, 48006])
key: edge_index
item,slices: tensor([[ 0,  1,  1,  ..., 11, 22, 17],
        [ 1,  0,  2,  ..., 16, 17, 22]]) tensor([     0,     24,     86,  ..., 100788, 100864, 100912])
key: edge_attr
item,slices: tensor([[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [3, 0],
        [3, 0],
        [3, 0]]) tensor([     0,     24,     86,  ..., 100788, 100864, 100912])
key: id
item,slices: tensor([   0,    1,    2,  ..., 1424, 1425, 1426]) tensor([   0,    1,    2,  ..., 1425, 1426, 1427])
++++++++++ Data(y=[27], x=[13, 2], edge_index=[2, 24], edge_attr=[24, 2], id=[1])


NameError: name 'SeqMolDataLoader' is not defined

In [27]:
if args.split == "scaffold":
        smiles_list = pd.read_csv('./dataset/' + args.dataset + '/processed/smiles.csv', header=None)[0].tolist()
        
        #print('smiles_list:',smiles_list)
        mol_train_dataset, mol_valid_dataset, mol_test_dataset ,seq_train_dataset,seq_valid_dataset,seq_test_dataset= scaffold_split_1(seq_gnn_dataset, smiles_list, null_value=0, frac_train=0.8,frac_valid=0.1, frac_test=0.1)##########dataset
        print("scaffold")
elif args.split == "random":
        train_dataset, valid_dataset, test_dataset = random_split(dataset, null_value=0, frac_train=0.8,frac_valid=0.1, frac_test=0.1, seed = args.seed)
        print("random")
elif args.split == "random_scaffold":
        smiles_list = pd.read_csv('./dataset/' + args.dataset + '/processed/smiles.csv', header=None)[0].tolist()
        train_dataset, valid_dataset, test_dataset = random_scaffold_split(dataset, smiles_list, null_value=0, frac_train=0.8,frac_valid=0.1, frac_test=0.1, seed = args.seed)
        print("random scaffold")
else:
        raise ValueError("Invalid split option.")

print('++++++++++', mol_train_dataset[0])

'''
seq_mol_train_dataset=SeqMolDataset(seq_train_dataset,mol_train_dataset)
print('seq_mol_train_dataset:',seq_mol_train_dataset)
seq_mol_valid_dataset=SeqMolDataset(seq_valid_dataset,mol_valid_dataset)
seq_mol_test_dataset=SeqMolDataset(seq_train_dataset,mol_test_dataset)

seq_train_dataset=seq_mol_train_dataset.datasetA
mol_train_dataset=seq_mol_train_dataset.datasetB
'''
seq_train_dataloader=DataLoader(seq_train_dataset,batch_size=args.batch_size,num_workers=args.num_workers,shuffle=False)
mol_train_dataloader=DataLoader(mol_train_dataset,batch_size=args.batch_size,num_workers=args.num_workers,shuffle=False)
print(type(seq_train_dataset),type(mol_train_dataset))


for i ,dataA in enumerate(seq_train_dataloader):
    print('DATA:',dataA)

'''
for i ,dataA in enumerate(seq_train_dataloader):
    print('DATA:',dataA)
'''    
'''
for i,(seq,mol) in enumerate(zip(seq_train_dataloader,mol_train_dataloader)):
    print(i,seq,mol)
'''

scaffold
key: y
item,slices: tensor([ 1,  1, -1,  ...,  1,  1,  1]) tensor([    0,    27,    54,  ..., 38475, 38502, 38529])
key: x
item,slices: tensor([[5, 0],
        [5, 0],
        [6, 0],
        ...,
        [5, 0],
        [5, 0],
        [5, 0]]) tensor([    0,    13,    42,  ..., 47949, 47983, 48006])
key: edge_index
item,slices: tensor([[ 0,  1,  1,  ..., 11, 22, 17],
        [ 1,  0,  2,  ..., 16, 17, 22]]) tensor([     0,     24,     86,  ..., 100788, 100864, 100912])
key: edge_attr
item,slices: tensor([[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [3, 0],
        [3, 0],
        [3, 0]]) tensor([     0,     24,     86,  ..., 100788, 100864, 100912])
key: id
item,slices: tensor([   0,    1,    2,  ..., 1424, 1425, 1426]) tensor([   0,    1,    2,  ..., 1425, 1426, 1427])
++++++++++ Data(y=[27], x=[13, 2], edge_index=[2, 24], edge_attr=[24, 2], id=[1])
<class 'pandas.core.series.Series'> loader.MoleculeDataset


KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/zhengfang/anaconda3/envs/new_pytorch/lib/python3.6/site-packages/pandas/core/indexes/base.py", line 2891, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1032, in pandas._libs.hashtable.Int64HashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1039, in pandas._libs.hashtable.Int64HashTable.get_item
KeyError: 1

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/zhengfang/anaconda3/envs/new_pytorch/lib/python3.6/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/zhengfang/anaconda3/envs/ne

In [24]:
def train(args, epoch, model, device, loader, optimizer):
    model.train()
    #epoch_iter = tqdm(loader, desc="Iteration")
    for step, (A,B) in enumerate(zip(loader.dataloaderA,loader.dataloaderB)):
        print('A:',A)
        print('B:',B)
        #A = A.to(device)
        B=B.to(device)
        #print('batch:',batch.x,batch.edge_index,batch.edge_attr,batch.batch)
        pred=model(A,*B)
        #pred, node_representation = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
        y = B.y.view(pred.shape).to(torch.float64)

        #Whether y is non-null or not.
        is_valid = y**2 > 0
        #Loss matrix
        loss_mat = criterion(pred.double(), (y+1)/2)
        #loss matrix after removing null target
        loss_mat = torch.where(is_valid, loss_mat, torch.zeros(loss_mat.shape).to(loss_mat.device).to(loss_mat.dtype))  
        optimizer.zero_grad()
        loss = torch.sum(loss_mat)/torch.sum(is_valid)
        loss.backward()
        optimizer.step()
        epoch_iter.set_description(f"Epoch: {epoch} tloss: {loss:.4f}")

        
        
def train1(args, epoch, model, device, loader, optimizer):
    model.train()
    #epoch_iter = tqdm(loader, desc="Iteration")
    for step, C in enumerate(loader):
        
        A=C.loaderA
        B=C.loaderB
        print('A:',C.loaderA)
        print('B:',C.loaderB)
        #A = A.to(device)
        B=B.to(device)
        #print('batch:',batch.x,batch.edge_index,batch.edge_attr,batch.batch)
        pred=model(A,*B)
        #pred, node_representation = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
        y = B.y.view(pred.shape).to(torch.float64)

        #Whether y is non-null or not.
        is_valid = y**2 > 0
        #Loss matrix
        loss_mat = criterion(pred.double(), (y+1)/2)
        #loss matrix after removing null target
        loss_mat = torch.where(is_valid, loss_mat, torch.zeros(loss_mat.shape).to(loss_mat.device).to(loss_mat.dtype))  
        optimizer.zero_grad()
        loss = torch.sum(loss_mat)/torch.sum(is_valid)
        loss.backward()
        optimizer.step()
        epoch_iter.set_description(f"Epoch: {epoch} tloss: {loss:.4f}")


In [26]:
train_acc_list = []
val_acc_list = []
test_acc_list = []

if not args.filename == "":
    fname = 'runs/seq_mol_finetune_cls_runseed' + str(args.runseed) + '/' + args.filename
    #delete the directory if there exists one
    if os.path.exists(fname):
        shutil.rmtree(fname)
        print("removed the existing file.")
    writer = SummaryWriter(fname)

for epoch in range(1, args.epochs+1):
    print("====epoch " + str(epoch))
        
    train(args, epoch, model, device, seq_mol_train_loader, optimizer)

    print("====Evaluation")
    if args.eval_train:
        train_acc = eval(args, model, device, train_loader)
    else:
        print("omit the training accuracy computation")
        train_acc = 0
    val_acc = eval(args, model, device, val_loader)
    test_acc = eval(args, model, device, test_loader)

    print("train: %f val: %f test: %f" %(train_acc, val_acc, test_acc))
    val_acc_list.append(val_acc)
    test_acc_list.append(test_acc)
    train_acc_list.append(train_acc)

    if not args.filename == "":
        writer.add_scalar('data/train auc', train_acc, epoch)
        writer.add_scalar('data/val auc', val_acc, epoch)
        writer.add_scalar('data/test auc', test_acc, epoch)

print('Best epoch:', val_acc_list.index(max(val_acc_list)))
print('Best auc: ', test_acc_list[val_acc_list.index(max(val_acc_list))])

exp_path = os.getcwd() + '/{}_results/{}/'.format(args.input_model_file, args.dataset)
if not os.path.exists(exp_path):
    os.makedirs(exp_path)

df = pd.DataFrame({'train':train_acc_list,'valid':val_acc_list,'test':test_acc_list})
df.to_csv(exp_path + 'seed{}.csv'.format(args.runseed))

logs = 'Dataset:{}, Seed:{}, Best Epoch:{}, Best Acc:{:.5f}'.format(args.dataset, args.runseed, val_acc_list.index(max(val_acc_list)), test_acc_list[val_acc_list.index(max(val_acc_list))])
with open(exp_path + '{}_log.csv'.format(args.dataset),'a+') as f:
    f.write('\n')
    f.write(logs)

if not args.filename == "":
    writer.close()

====epoch 1


KeyError: 1